## 🏠 **Las ventajas de jugar en casa**
---

### ⚙️ Native Executive Engine

Motor de ejecución vectorizado que sustituye la ejecución tradicional de Java por código nativo en C++ para maximizar el rendimiento.

Su función principal es procesar las tareas de Spark directamente en el hardware moderno, eliminando la sobrecarga de la JVM y la serialización para acelerar drásticamente las consultas y transformaciones de datos.

En escenarios compatibles, se habla de mejoras de un 2x-5x.

In [ ]:
spark.conf.set("spark.native.enabled", "true")

### 🛠️ Optimizando, que es gerundio

Disponemos de diferentes propiedades que facilitan el ajuste y tunning de operaciones de Spark de forma automática.

Microsoft recomienda activar algunas de ellas, pero esto no significa que sea activarlas y ya.

#### ⚡Fast optimize

Analiza de forma inteligente los archivos de una tabla Delta y omite las operaciones de compactación que no mejoren el rendimiento de forma significativa cuando ejecutamos un comando OPTIMIZE explícito.

En lugar de compactar archivos ciegamente cada vez que existen archivos pequeños, Fast Optimize evalúa si cada grupo de archivos candidato cumple los objetivos de compactación recomendados. De lo contrario, omite ese grupo o reduce el número de archivos que compacta.

In [ ]:
spark.conf.set('spark.microsoft.delta.optimize.fast.enabled', True)

#### 🗜️ File-level compaction targets

Para evitar la recompactación de archivos ya compactados. 

Cuando se habilita, los archivos no se recompactan si anteriormente cumplían al menos la mitad del tamaño objetivo del archivo en el momento que se compactaron. Especialmente útil ya que el tamaño objetivo de compactación puede cambiar con el tiempo.

In [ ]:
spark.conf.set('spark.microsoft.delta.optimize.fileLevelTarget.enabled', True)

## 📦 **Vienen ya en la caja**
---

Propiedades inherentes de Spark que también se recomienda activar/tener en cuenta.

Pueden activarse tanto a nivel de sesión como a nivel de tabla.

#### 🅰️ Auto compaction
Evalúa el estado después de cada operación de escritura. 

Cuando detecta demasiados archivos pequeños, ejecuta una operación OPTIMIZE inmediatamente después de que se confirme la escritura. Este enfoque es óptimo porque la compactación solo se ejecuta cuando se determina que es beneficioso. Microsoft recomienda activar este ajuste frente a trabajos programados de mantenimiento (OPTIMIZE), ahora esto siempre que la latencia adicional que supone sea asumible por el proceso relacionado.

Resulta muy beneficioso usarlo en conjunción con la siguiente.

**OJO** No elimina la necesidad de uso del comando OPTIMIZE en todos los escenarios.

In [ ]:
spark.conf.set('spark.databricks.delta.autoCompact.enabled', True)

#### ✍🏻 Optimize write

Reduce la sobrecarga de los archivos pequeños mediante la compactación previa a la escritura, lo que genera menos archivos y de mayor tamaño. Este enfoque mezcla los datos en memoria antes de que Spark escriba, lo que maximiza el potencial de generar archivos de tamaño adecuado sin necesidad de operaciones de compactación tras la escritura.

No obstante, debe utilizarse con precaución, ya que su coste computacional puede añadir un tiempo de procesamiento excesivo e innecesario. Principalmente es beneficiosa cuando una operación de escritura produciría archivos pequeños que serían candidatos para una compactación posterior.

Muy adecuada para:
- Tablas particionadas
- Tablas con inserciones pequeñas frecuentes
- Operaciones que probablemente afecten a muchos archivos (MERGE, UPDATE y DELETE)

In [ ]:
spark.conf.set('spark.databricks.delta.optimizeWrite.enabled', True)